# Import necessary libraries

In [2]:
import pandas as pd
from datetime import date
import holidays

# Load data

In [7]:
data = pd.read_csv('../raw/data_raw.csv',encoding='latin1')
item_master_data = pd.read_csv('../raw/item_master.csv', encoding='latin1')

# Drop unnecessary columns


In [49]:
if(len(data.columns) == 17):
    data.drop(['Branch', 'Schm.Disc ', 'Bill Disc ', 'Doctor Name ', 'Patient Name ', 'Scheduler ', 'Batch No '], axis=1, inplace=True)

# Rename necessary common column

In [50]:
data.rename(columns={'Prod Code': 'Product Code', ' ': 'Customer ID', 'Prod Name': 'Product Name'}, inplace=True)

# Merge Dataframes to get subcat name

In [51]:
merged_df = pd.merge(data, item_master_data[['Product Code', 'SUB CAT NAME', 'CATEGORY NAME']], on='Product Code', how='inner')
merged_df.rename(columns={'SUB CAT NAME': 'Sub Category Name', 'CATEGORY NAME': 'Category Name'}, inplace=True)

# Remove NA customers and 1 Customers and convert column data types.

In [52]:
merged_df.dropna(subset=['Customer ID'], inplace=True)
mergded_df = merged_df[merged_df['Customer ID'] != 1]
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Customer ID'] = merged_df['Customer ID'].astype(int)
merged_df['Cost Rate'] = merged_df['Cost Rate'].round(2)

# Add necessary/requried columns 

In [53]:
try:
    merged_df.insert(1, 'Day', merged_df['Date'].dt.day_name())
    merged_df.insert(2, 'Month', merged_df['Date'].dt.strftime('%B'))
    merged_df.insert(3, 'Season', ' ')
    merged_df.insert(4, 'Festival', ' ')
    merged_df.insert(12, 'Margin', (merged_df['Sale Rate'] - merged_df['Cost Rate']).round(2))
except Exception:
    pass

# Seasons transformation

In [54]:
season = []
for x in merged_df['Date']:
    month = x.month
    if(month in [2, 3]):
        season.append('Spring')
    elif(month in [4, 5, 6]):
        season.append('Summer')
    elif(month in [7, 8, 9]):
        season.append('Monsoon')
    elif(month in [10, 11]):
        season.append('Autumn')
    else:
        season.append('Winter')
merged_df['Season'] = season

# Festival/Holiday Transformation

In [55]:
unique_years = merged_df['Date'].dt.year.unique().tolist()
holidays_dict = dict()
holidays_list = []
for date, name in sorted(holidays.IN(years=tuple(unique_years)).items()):
    holidays_dict[str(date)] = name

for x in merged_df['Date']:
    if(str(x)[:10] in holidays_dict):
        holidays_list.append(holidays_dict[str(x)[:10]])
    else:
        holidays_list.append('Normal Day')
merged_df['Festival'] = holidays_list

# CSV conversion

In [56]:
merged_df.to_csv('Transformed_data.csv')
merged_df

,Date,Day,Month,Season,Festival,Bill No,Product Code,Product Name,Qty,Cost Rate,Sale Rate,MRP,Margin,Amount,Customer ID,Sub Category Name,Category Name
3,2018-03-01,Thursday,March,Spring,Normal Day,136210,7402,AMUL FRESH MILK GOLD PP 500ML,2.0,25.20,26.0,26.0,0.80,52.0,1996,DAIRY,DAIRY & FROZEN
12,2018-03-02,Friday,March,Spring,Holi,136639,7402,AMUL FRESH MILK GOLD PP 500ML,1.0,25.20,26.0,26.0,0.80,26.0,2014,DAIRY,DAIRY & FROZEN
15,2018-03-02,Friday,March,Spring,Holi,136707,7402,AMUL FRESH MILK GOLD PP 500ML,1.0,25.20,26.0,26.0,0.80,26.0,2075,DAIRY,DAIRY & FROZEN
25,2018-03-03,Saturday,March,Spring,Normal Day,137325,7402,AMUL FRESH MILK GOLD PP 500ML,2.0,25.20,26.0,26.0,0.80,52.0,2293,DAIRY,DAIRY & FROZEN
32,2018-03-03,Saturday,March,Spring,Normal Day,137537,7402,AMUL FRESH MILK GOLD PP 500ML,1.0,25.20,26.0,26.0,0.80,26.0,1839,DAIRY,DAIRY & FROZEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894624,2017-04-25,Tuesday,April,Summer,Normal Day,9874,13502,AYR CLD WAX LMNVNGSGR 150GM,1.0,33.40,40.0,40.0,6.60,40.0,526,WAXING KAJAL LIP,COSMETIC & BEAUTY
894625,2017-04-26,Wednesday,April,Summer,Normal Day,10043,13213,BLUEBIRD ICRM MX RYLNUT 100GM,1.0,43.21,54.0,54.0,10.79,54.0,1371,BAKING,JAM/SAUCE/ SYRUP/SPREAD
894628,2017-04-28,Friday,April,Summer,Normal Day,11036,5805,PARAMPARA MTR PANEER 80 GM,1.0,38.37,48.0,48.0,9.63,48.0,1807,MASALA PASTES,SPICES & MASALAS
894629,2017-04-29,Saturday,April,Summer,Normal Day,11279,12927,BARBIE TLC SPRING ROSE 200GM,1.0,75.01,100.0,100.0,24.99,100.0,975,DEO & ANTIPERSPIRANTS,COSMETIC & BEAUTY
